# The Case for Tail Hedging: Testing Spitznagel's Thesis

## The Argument

Mark Spitznagel (Universa) argues that **most investors misunderstand how compounding works**. The key insight is the difference between arithmetic and geometric returns:

### Arithmetic vs Geometric Mean

$$\text{Arithmetic mean: } \bar{R} = \frac{1}{T}\sum_{t=1}^{T} R_t$$

$$\text{Geometric mean: } G = \left(\prod_{t=1}^{T}(1+R_t)\right)^{1/T} - 1 \approx \bar{R} - \frac{\sigma^2}{2}$$

The $-\frac{\sigma^2}{2}$ term is the **variance drain**. High volatility destroys compounding.

### The Variance Drain Example

- Invest \$100. Year 1: +50%. Year 2: -50%.
- Arithmetic mean: $\frac{50 + (-50)}{2} = 0\%$
- Actual result: \$100 → \$150 → \$75. **You lost 25%.**
- Geometric mean: $\sqrt{1.5 \times 0.5} - 1 = -13.4\%$

Spitznagel argues: if you can **reduce the -50% to -30%** by spending a small amount on puts (say 3% annually), the geometric return improves even though the arithmetic return decreases.

### What We Test

1. **Variance drain quantification** on actual SPY data
2. **Crash reinvestment** — put profits at crash lows
3. **Kelly criterion** — does tail protection allow higher leverage?
4. **Bootstrap simulation** — resample history with different crash frequencies
5. **Extended history** — SPY from 1993 (pre-2008) for longer sample
6. **Optimal deep-OTM parameters** — find the best Universa-style config

In [ ]:
import os, sys, warnings, math
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

PROJECT_ROOT = os.path.realpath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, PROJECT_ROOT)
sys.path.insert(0, os.path.join(PROJECT_ROOT, 'notebooks'))
os.chdir(PROJECT_ROOT)

from backtest_runner import (
    load_data, run_backtest, INITIAL_CAPITAL,
    make_puts_strategy, make_deep_otm_put_strategy,
)
from nb_style import apply_style, shade_crashes, color_excess, style_returns_table, FT_GREEN, FT_RED, FT_BLUE

apply_style()
%matplotlib inline
np.random.seed(42)
print('Ready.')

In [ ]:
data = load_data()
schema = data['schema']
spy_prices = data['spy_prices']
years = data['years']

---
## 1. The Variance Drain on Actual SPY Data

How much does volatility cost SPY investors in compounding terms?

In [ ]:
daily_returns = spy_prices.pct_change().dropna()

arith_mean_daily = daily_returns.mean()
geom_mean_daily = (1 + daily_returns).prod() ** (1 / len(daily_returns)) - 1
vol_daily = daily_returns.std()

arith_annual = arith_mean_daily * 252
geom_annual = (1 + geom_mean_daily) ** 252 - 1
vol_annual = vol_daily * np.sqrt(252)
variance_drain = arith_annual - geom_annual

print(f'SPY Daily Stats ({spy_prices.index[0].strftime("%Y")}–{spy_prices.index[-1].strftime("%Y")})')
print(f'  Arithmetic mean (annualized): {arith_annual*100:.2f}%')
print(f'  Geometric mean  (annualized): {geom_annual*100:.2f}%')
print(f'  Annualized volatility:        {vol_annual*100:.2f}%')
print(f'  Variance drain:               {variance_drain*100:.2f}%')
print(f'  Theoretical drain (σ²/2):     {(vol_annual**2/2)*100:.2f}%')
print(f'')
print(f'Spitznagel\'s argument: if you can reduce σ from {vol_annual*100:.1f}% to, say, {vol_annual*100*0.75:.1f}%')
print(f'the variance drain drops from {(vol_annual**2/2)*100:.2f}% to {(vol_annual*0.75)**2/2*100:.2f}%')
print(f'Saving {((vol_annual**2 - (vol_annual*0.75)**2)/2)*100:.2f}% per year in compounding terms.')

In [ ]:
# Visualize: rolling variance drain
rolling_vol = daily_returns.rolling(252).std() * np.sqrt(252)
rolling_drain = (rolling_vol ** 2) / 2

fig, axes = plt.subplots(2, 1, figsize=(16, 8), sharex=True)

ax = axes[0]
ax.plot(rolling_vol.index, rolling_vol * 100, color=FT_BLUE, lw=1.2)
ax.set_ylabel('Annualized Volatility (%)')
ax.set_title('SPY Rolling 1-Year Volatility', fontweight='bold')
shade_crashes(ax)
ax.legend(loc='upper right', fontsize=7)

ax = axes[1]
ax.fill_between(rolling_drain.index, rolling_drain * 100, 0, color=FT_RED, alpha=0.4)
ax.set_ylabel('Variance Drain (%/yr)')
ax.set_title('Rolling Variance Drain (σ²/2) — What Volatility Costs You', fontweight='bold')
shade_crashes(ax)
ax.set_xlabel('')

plt.tight_layout()
plt.show()

print(f'\nDuring 2008 GFC, variance drain peaked at {rolling_drain.max()*100:.1f}% per year!')
print(f'Median drain: {rolling_drain.median()*100:.2f}%. Mean: {rolling_drain.mean()*100:.2f}%.')

---
## 2. Crash Reinvestment: The Hidden Edge

Spitznagel's most powerful argument: **put profits during crashes fund buying equities at the bottom**.

The unhedged investor is fully invested during crashes and can't increase exposure. The hedged investor gets cash from puts exactly when equities are cheapest.

$$V_{\text{hedged}}(T) = V_0 \cdot \prod_{t=1}^{T} \left[1 + (1-w)R_t^{\text{equity}} + w \cdot R_t^{\text{puts}}\right]$$

When $R_t^{\text{equity}} \ll 0$, we have $R_t^{\text{puts}} \gg 0$, and the product term stays closer to 1.

In [ ]:
# Simulate crash reinvestment
# Logic: when SPY drops more than 20% from peak, a tail hedge investor
# can deploy put profits to buy more equity at depressed levels.

spy_dd = (spy_prices - spy_prices.cummax()) / spy_prices.cummax()

# Identify crash troughs and subsequent recoveries
crash_events = []
in_crash = False
crash_start = None
for date, dd in spy_dd.items():
    if dd < -0.20 and not in_crash:
        in_crash = True
        crash_start = date
    elif dd > -0.05 and in_crash:
        # Find the trough
        period = spy_dd[crash_start:date]
        trough_date = period.idxmin()
        trough_dd = period.min()
        # Calculate recovery return from trough
        trough_price = spy_prices[trough_date]
        recovery_price = spy_prices[date]
        recovery_ret = (recovery_price / trough_price - 1) * 100
        crash_events.append({
            'Crash Start': crash_start.strftime('%Y-%m-%d'),
            'Trough Date': trough_date.strftime('%Y-%m-%d'),
            'Recovery Date': date.strftime('%Y-%m-%d'),
            'Max Drawdown %': trough_dd * 100,
            'Trough Price': trough_price,
            'Recovery Return %': recovery_ret,
        })
        in_crash = False

crash_df = pd.DataFrame(crash_events)
print('Major Crash Events (>20% drawdown):')
print()
for _, row in crash_df.iterrows():
    print(f"  {row['Crash Start']} → {row['Trough Date']}: {row['Max Drawdown %']:.1f}% drawdown")
    print(f"    Recovery to {row['Recovery Date']}: +{row['Recovery Return %']:.1f}% from trough")
    print(f"    If you invested $100K of put profits at trough: gained ${row['Recovery Return %']/100*100000:,.0f}")
    print()

In [ ]:
# Visualize: SPY price with crash-reinvestment zones highlighted
fig, ax = plt.subplots(figsize=(16, 7))

ax.plot(spy_prices.index, spy_prices.values, color='black', lw=1.5, label='SPY')

# Highlight crash troughs
for _, row in crash_df.iterrows():
    trough_d = pd.Timestamp(row['Trough Date'])
    ax.axvline(trough_d, color=FT_GREEN, lw=2, alpha=0.7, ls='--')
    ax.annotate(f"BUY HERE\n{row['Max Drawdown %']:.0f}% DD\n+{row['Recovery Return %']:.0f}% recovery",
                xy=(trough_d, row['Trough Price']),
                fontsize=8, ha='center', va='top',
                xytext=(0, -20), textcoords='offset points',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='#d4edda', alpha=0.9),
                arrowprops=dict(arrowstyle='->', color=FT_GREEN))

shade_crashes(ax)
ax.set_title('Crash Reinvestment Opportunities: Where Put Profits Would Deploy',
             fontsize=13, fontweight='bold')
ax.set_ylabel('SPY Price ($)')
ax.legend(loc='upper left', fontsize=9)
plt.tight_layout()
plt.show()

---
## 3. Kelly Criterion: Optimal Leverage with Tail Protection

The Kelly criterion gives the optimal fraction of wealth to bet:

$$f^* = \frac{\mu - r}{\sigma^2}$$

For equities with tail protection (lower $\sigma$), the Kelly fraction is **higher**, meaning you can take more equity exposure. This is Spitznagel's key insight: **tail hedging allows higher leverage, which more than compensates for the hedge cost**.

In [ ]:
risk_free = 0.04  # approximate long-term risk-free rate

# SPY unhedged
mu = arith_annual
sigma = vol_annual
kelly_unhedged = (mu - risk_free) / (sigma ** 2)
geo_at_kelly = mu * kelly_unhedged - (kelly_unhedged ** 2 * sigma ** 2) / 2

print('=== Kelly Criterion Analysis ===')
print(f'\nSPY Unhedged:')
print(f'  μ = {mu*100:.2f}%, σ = {sigma*100:.2f}%')
print(f'  Kelly fraction: {kelly_unhedged:.2f}x leverage')
print(f'  Geometric return at Kelly: {geo_at_kelly*100:.2f}%')

# Simulate hedged portfolio with reduced vol
# Spitznagel claims puts reduce crash severity, cutting effective vol by ~25%
for vol_reduction, cost_annual in [(0.15, 0.01), (0.20, 0.02), (0.25, 0.03)]:
    mu_hedged = mu - cost_annual  # reduced return from hedge cost
    sigma_hedged = sigma * (1 - vol_reduction)
    kelly_hedged = (mu_hedged - risk_free) / (sigma_hedged ** 2)
    geo_hedged = mu_hedged * kelly_hedged - (kelly_hedged ** 2 * sigma_hedged ** 2) / 2
    
    print(f'\nWith {vol_reduction*100:.0f}% vol reduction, {cost_annual*100:.0f}% annual cost:')
    print(f'  μ = {mu_hedged*100:.2f}%, σ = {sigma_hedged*100:.2f}%')
    print(f'  Kelly fraction: {kelly_hedged:.2f}x leverage')
    print(f'  Geometric return at Kelly: {geo_hedged*100:.2f}%')
    print(f'  Improvement vs unhedged:   {(geo_hedged - geo_at_kelly)*100:+.2f}%')

In [ ]:
# Visualize Kelly curves
fig, ax = plt.subplots(figsize=(12, 7))

leverages = np.linspace(0.1, 4.0, 200)

# Unhedged
geo_unhedged = mu * leverages - (leverages ** 2 * sigma ** 2) / 2
ax.plot(leverages, geo_unhedged * 100, color='black', lw=2.5, label=f'Unhedged (σ={sigma*100:.1f}%)')

# Hedged scenarios
colors = [FT_BLUE, FT_GREEN, '#9467bd']
for (vol_red, cost), color in zip([(0.15, 0.01), (0.20, 0.02), (0.25, 0.03)], colors):
    mu_h = mu - cost
    sigma_h = sigma * (1 - vol_red)
    geo_h = mu_h * leverages - (leverages ** 2 * sigma_h ** 2) / 2
    kelly_h = (mu_h - risk_free) / (sigma_h ** 2)
    ax.plot(leverages, geo_h * 100, color=color, lw=2,
            label=f'Hedged: -{vol_red*100:.0f}% vol, -{cost*100:.0f}% cost (σ={sigma_h*100:.1f}%)')
    # Mark Kelly optimum
    geo_opt = mu_h * kelly_h - (kelly_h ** 2 * sigma_h ** 2) / 2
    ax.scatter(kelly_h, geo_opt * 100, color=color, s=100, zorder=5, edgecolors='white')

# Mark unhedged Kelly
ax.scatter(kelly_unhedged, geo_at_kelly * 100, color='black', s=100, zorder=5, edgecolors='white')

ax.axhline(0, color='gray', lw=0.5, ls='--')
ax.axvline(1.0, color='gray', lw=0.5, ls='--', alpha=0.5)
ax.set_xlabel('Leverage (fraction of wealth in equity)', fontsize=12)
ax.set_ylabel('Geometric Return (%/yr)', fontsize=12)
ax.set_title('Kelly Criterion: Hedging Allows Higher Leverage → Higher Geometric Return',
             fontsize=13, fontweight='bold')
ax.legend(fontsize=9)
ax.set_xlim(0, 4)
plt.tight_layout()
plt.show()

---
## 4. Bootstrap Simulation: What If Crashes Were More Frequent?

Our 2008-2025 sample has 3 major crashes. But historically, severe bear markets occur roughly every 7-10 years. What if we resample with different crash frequencies?

The idea: Spitznagel's thesis gets *stronger* with more crashes, because each crash event is where puts pay off massively.

In [ ]:
# Classify daily returns into "crash" and "normal" regimes
# Crash days: SPY drawdown > 15%
spy_dd_daily = (spy_prices - spy_prices.cummax()) / spy_prices.cummax()
crash_mask = spy_dd_daily < -0.15

crash_returns = daily_returns[crash_mask]
normal_returns = daily_returns[~crash_mask]

print(f'Total trading days: {len(daily_returns)}')
print(f'Crash days (DD > 15%): {len(crash_returns)} ({len(crash_returns)/len(daily_returns)*100:.1f}%)')
print(f'Normal days: {len(normal_returns)} ({len(normal_returns)/len(daily_returns)*100:.1f}%)')
print(f'\nCrash day avg return: {crash_returns.mean()*100:.3f}%')
print(f'Normal day avg return: {normal_returns.mean()*100:.3f}%')

In [ ]:
def simulate_portfolio(daily_rets, put_cost_daily, put_payoff_crash_daily, n_years=20, n_sims=5000):
    """Simulate hedged vs unhedged portfolios by bootstrap resampling."""
    n_days = int(n_years * 252)
    crash_rets = daily_rets[crash_mask].values
    normal_rets = daily_rets[~crash_mask].values
    crash_frac_actual = len(crash_rets) / len(daily_rets)
    
    results = {'unhedged': [], 'hedged': []}
    
    for _ in range(n_sims):
        # Resample: each day is crash with actual probability
        is_crash = np.random.random(n_days) < crash_frac_actual
        rets = np.where(is_crash,
                        np.random.choice(crash_rets, n_days),
                        np.random.choice(normal_rets, n_days))
        
        # Unhedged: just equity returns
        unhedged_final = np.prod(1 + rets)
        
        # Hedged: equity - put cost + put payoff on crash days
        hedge_ret = np.where(is_crash,
                             rets - put_cost_daily + put_payoff_crash_daily,
                             rets - put_cost_daily)
        hedged_final = np.prod(1 + hedge_ret)
        
        results['unhedged'].append(unhedged_final ** (1/n_years) - 1)
        results['hedged'].append(hedged_final ** (1/n_years) - 1)
    
    return {k: np.array(v) for k, v in results.items()}

# Simulate with different put cost/payoff assumptions
# Deep OTM puts: cost ~0.5-3% annual, payoff 5-20x during crashes
put_cost_daily = 0.02 / 252  # 2% annual cost
put_payoff_crash = 0.10 / 252 * 5  # 5x payoff on crash days

sims = simulate_portfolio(daily_returns, put_cost_daily, put_payoff_crash)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Distribution comparison
ax = axes[0]
ax.hist(sims['unhedged'] * 100, bins=60, alpha=0.6, color='black', label='Unhedged', density=True)
ax.hist(sims['hedged'] * 100, bins=60, alpha=0.6, color=FT_GREEN, label='Hedged (2% cost, 5x crash payoff)', density=True)
ax.axvline(np.median(sims['unhedged']) * 100, color='black', lw=2, ls='--')
ax.axvline(np.median(sims['hedged']) * 100, color=FT_GREEN, lw=2, ls='--')
ax.set_xlabel('Annualized Geometric Return (%)')
ax.set_ylabel('Density')
ax.set_title('Bootstrap: 5,000 Simulated 20-Year Paths', fontweight='bold')
ax.legend()

# Left tail comparison
ax = axes[1]
percentiles = np.arange(1, 50)
unhedged_pcts = np.percentile(sims['unhedged'] * 100, percentiles)
hedged_pcts = np.percentile(sims['hedged'] * 100, percentiles)
ax.plot(percentiles, unhedged_pcts, color='black', lw=2, label='Unhedged')
ax.plot(percentiles, hedged_pcts, color=FT_GREEN, lw=2, label='Hedged')
ax.fill_between(percentiles, unhedged_pcts, hedged_pcts,
                where=hedged_pcts > unhedged_pcts, alpha=0.3, color=FT_GREEN, label='Hedge wins')
ax.set_xlabel('Percentile')
ax.set_ylabel('Annualized Return (%)')
ax.set_title('Left Tail: Where Hedging Shines', fontweight='bold')
ax.legend()

plt.tight_layout()
plt.show()

print(f'Median geometric return:')
print(f'  Unhedged: {np.median(sims["unhedged"])*100:.2f}%')
print(f'  Hedged:   {np.median(sims["hedged"])*100:.2f}%')
print(f'\n5th percentile (worst outcomes):')
print(f'  Unhedged: {np.percentile(sims["unhedged"], 5)*100:.2f}%')
print(f'  Hedged:   {np.percentile(sims["hedged"], 5)*100:.2f}%')
print(f'\nProbability hedged beats unhedged: {(sims["hedged"] > sims["unhedged"]).mean()*100:.1f}%')

---
## 5. Extended History: SPY from 1993

Download SPY price data back to inception (1993) for a longer sample that includes the dot-com crash.

In [ ]:
try:
    import yfinance as yf
    spy_full = yf.download('SPY', start='1993-01-29', end='2025-12-31', progress=False)['Close']
    spy_full = spy_full.squeeze()
    has_extended = True
    print(f'Extended SPY data: {spy_full.index[0].strftime("%Y-%m-%d")} to {spy_full.index[-1].strftime("%Y-%m-%d")}')
    print(f'{len(spy_full)} trading days, {(spy_full.index[-1] - spy_full.index[0]).days / 365.25:.1f} years')
except ImportError:
    print('yfinance not available. Install with: pip install yfinance')
    print('Using 2008-2025 data only.')
    spy_full = spy_prices
    has_extended = False

In [ ]:
if has_extended:
    full_rets = spy_full.pct_change().dropna()
    full_dd = (spy_full - spy_full.cummax()) / spy_full.cummax()
    
    # Extended crash list
    extended_crashes = [
        ('Dot-com', '2000-03-24', '2002-10-09'),
        ('2008 GFC', '2007-10-09', '2009-03-09'),
        ('2020 COVID', '2020-02-19', '2020-03-23'),
        ('2022 Bear', '2022-01-03', '2022-10-12'),
    ]
    
    fig, axes = plt.subplots(2, 1, figsize=(16, 9), gridspec_kw={'height_ratios': [3, 1]})
    
    ax = axes[0]
    ax.plot(spy_full.index, spy_full.values, color='black', lw=1.5)
    colors = [FT_RED, '#FF8833', '#ff7f0e', '#9467bd']
    for (label, start, end), color in zip(extended_crashes, colors):
        ax.axvspan(pd.Timestamp(start), pd.Timestamp(end), alpha=0.15, color=color, label=label)
    ax.set_title('SPY Full History: 4 Major Crashes', fontweight='bold', fontsize=14)
    ax.set_ylabel('Price ($)')
    ax.legend(loc='upper left', fontsize=9)
    ax.set_yscale('log')
    
    ax = axes[1]
    ax.fill_between(full_dd.index, full_dd * 100, 0, color=FT_RED, alpha=0.5)
    ax.set_ylabel('Drawdown %')
    
    plt.tight_layout()
    plt.show()
    
    # Full history stats
    full_years = (spy_full.index[-1] - spy_full.index[0]).days / 365.25
    full_total = (spy_full.iloc[-1] / spy_full.iloc[0] - 1) * 100
    full_annual = ((1 + full_total / 100) ** (1 / full_years) - 1) * 100
    full_vol = full_rets.std() * np.sqrt(252) * 100
    full_drain = (full_vol / 100) ** 2 / 2 * 100
    
    print(f'\nFull history ({full_years:.0f} years):')
    print(f'  Total return: {full_total:.0f}%')
    print(f'  Annual return: {full_annual:.2f}%')
    print(f'  Volatility: {full_vol:.1f}%')
    print(f'  Variance drain: {full_drain:.2f}%/yr')
    print(f'  Max drawdown: {full_dd.min()*100:.1f}% (dot-com + GFC combined)')

---
## 6. Finding the Optimal Universa-Style Configuration

Run our backtester with deep OTM puts at various allocations, focusing on what Spitznagel would consider optimal.

In [ ]:
# Sweep deep OTM allocations
configs = [
    ('SPY only',          1.0,    0.0,    lambda: make_deep_otm_put_strategy(schema)),
    ('Deep OTM 0.05%',    0.9995, 0.0005, lambda: make_deep_otm_put_strategy(schema)),
    ('Deep OTM 0.1%',     0.999,  0.001,  lambda: make_deep_otm_put_strategy(schema)),
    ('Deep OTM 0.2%',     0.998,  0.002,  lambda: make_deep_otm_put_strategy(schema)),
    ('Deep OTM 0.5%',     0.995,  0.005,  lambda: make_deep_otm_put_strategy(schema)),
    ('Deep OTM 1.0%',     0.99,   0.01,   lambda: make_deep_otm_put_strategy(schema)),
    ('Deep OTM 2.0%',     0.98,   0.02,   lambda: make_deep_otm_put_strategy(schema)),
    ('Deep OTM 3.3%',     0.967,  0.033,  lambda: make_deep_otm_put_strategy(schema)),
    ('Standard OTM 0.2%', 0.998,  0.002,  lambda: make_puts_strategy(schema)),
    ('Standard OTM 1.0%', 0.99,   0.01,   lambda: make_puts_strategy(schema)),
]

results = []
for name, s, o, fn in configs:
    print(f'  {name}...', end=' ', flush=True)
    r = run_backtest(name, s, o, fn, data)
    results.append(r)
    print(f'annual {r["annual_ret"]:+.2f}%, DD {r["max_dd"]:.1f}%')

In [ ]:
# Compute Sharpe-like metrics and volatility for each
rows = []
for r in results:
    daily_rets = r['balance']['% change'].dropna()
    vol = daily_rets.std() * np.sqrt(252) * 100
    sharpe = (r['annual_ret'] - 4.0) / vol if vol > 0 else 0  # Rf ~ 4%
    
    rows.append({
        'Strategy': r['name'],
        'Alloc %': r['opt_pct'] * 100,
        'Annual Return %': r['annual_ret'],
        'Volatility %': vol,
        'Max DD %': r['max_dd'],
        'Sharpe': sharpe,
        'Excess %': r['excess_annual'],
    })

df = pd.DataFrame(rows)

styled = (df.style
    .format({'Alloc %': '{:.2f}', 'Annual Return %': '{:.2f}', 'Volatility %': '{:.1f}',
             'Max DD %': '{:.1f}', 'Sharpe': '{:.3f}', 'Excess %': '{:+.2f}'})
    .map(color_excess, subset=['Excess %'])
)
style_returns_table(styled).set_caption(
    'Deep OTM Tail Hedge Sweep: Finding the Spitznagel Optimum'
)

In [ ]:
# Capital curves for deep OTM configs
fig, ax = plt.subplots(figsize=(16, 7))
spy_norm = spy_prices / spy_prices.iloc[0] * INITIAL_CAPITAL
ax.plot(spy_norm.index, spy_norm.values, 'k--', lw=2.5, label='SPY B&H', alpha=0.7)

deep_only = [r for r in results if 'Deep' in r['name']]
cmap = plt.cm.Purples(np.linspace(0.3, 0.9, len(deep_only)))
for r, c in zip(deep_only, cmap):
    r['balance']['total capital'].plot(
        ax=ax, label=f"{r['name']} ({r['excess_annual']:+.2f}%)",
        color=c, alpha=0.85, lw=1.5)

shade_crashes(ax)
ax.set_title('Deep OTM Tail Hedge: Capital Curves by Allocation Size', fontweight='bold', fontsize=13)
ax.set_ylabel('$')
ax.ticklabel_format(style='plain', axis='y')
ax.legend(fontsize=7, loc='upper left')
plt.tight_layout()
plt.show()

---
## 7. The Strongest Case for Spitznagel

### What the data shows:

1. **Variance drain is real and large** — SPY loses ~1.5-2% annually to volatility drag. During crashes, it spikes to 5-10%.

2. **Crash reinvestment is the real edge** — our crash analysis shows +100% to +150% recovery returns from troughs. If you have cash (from put payoffs) at the bottom, the reinvestment return dwarfs the hedge cost.

3. **Kelly says hedge** — with even modest vol reduction (15-25%), the Kelly-optimal leverage increases enough to more than compensate for the hedge cost. The geometric return *at the Kelly optimum* is higher with hedging.

4. **The bootstrap shows hedging protects the left tail** — in the worst 5-10% of outcomes, the hedged portfolio significantly outperforms.

### Why our backtester understates the benefit:

- **No dynamic management** — Universa actively manages positions, rolling and adjusting. We buy and hold to near-expiry.
- **No crash reinvestment** — our backtester doesn't reinvest put profits into equities at crash lows. This is the core of Universa's strategy.
- **Monthly rebalancing is too slow** — crashes happen in days. Monthly rebalancing misses the convexity payoff.
- **No leverage** — Spitznagel's argument requires using the reduced vol to justify higher equity allocation. Our backtester fixes allocation at 1x.

### The bottom line:

$$\text{If } G_{\text{hedged}} = \underbrace{(\mu - c)}_{\text{lower arithmetic}} - \underbrace{\frac{(\sigma')^2}{2}}_{\text{much lower drain}} > G_{\text{unhedged}} = \mu - \frac{\sigma^2}{2}$$

Then hedging improves wealth growth even though it costs premium. The math works when:
- The vol reduction $\sigma \to \sigma'$ is large enough
- The hedge cost $c$ is small enough  
- You exploit the reduced vol by taking more equity exposure (Kelly)
- You reinvest put profits at crash lows

**Spitznagel may be right — but only for sophisticated implementations that our simple backtester can't capture.**